# Volume 3: Discrete Hidden Markov Models
    Jane Emeline Slagle
    Section 1 and done
    2/23/23

In [1]:
import numpy as np
import copy
import string
import codecs

In [2]:
class HMM(object):
    """
    Finite state space hidden markov model.
    """
    
    # Problem 1
    def __init__(self):
        """
        Initialize an HMM with parameters A, B, and pi, initialized as 
        None objects.
        """
        self.A = None
        self.B = None
        self.pi = None
    
    # Problem 2
    def _forward(self, z):
        """
        Compute the scaled forward probability matrix and scaling factors.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        
        Returns
        -------
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix
        
        c : ndarray of shape (T,)
            The scaling factors c = [c_0, c_1, ..., c_{T-1}]
        """
        #follow pseudo code for fward pass algorithm in lab manual:
        
        #get all initial values will need:
        T = np.shape(z)[0]     #told T is sequence of observations, observations are z here
        a = np.ones((T, len(self.pi)))  #a is shape of (T, n), len of pi is n here
        c = np.ones(T)         #c has shape of (T,)
        
        #get 1st values of alpha, c: this is lines 3-5 in pseudo code given
        a[0] = self.pi*self.B[z[0]]
        c[0] = 1/(np.sum(a[0]))
        a[0] = c[0] * a[0]
        
        #now do for loop through all T values like line 6 of pseudo code:
        for t in range(1, T):
            a[t] = np.sum(a[t-1]*self.A, axis = 1)*self.B[z[t]]  #compute a_t
            c[t] = 1/np.sum(a[t])                                #compute c_t
            a[t] = c[t]*a[t]                                     #rescale a_t
            
        return a, c  #want return alpha and c
    
    # Problem 3
    def _backward(self, z, c):
        """
        Compute the scaled backward probability matrix.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix
        """
        #follow bward pass pseudo code given:
        
        #get all initial values will need, for T, n and B:
        T = np.shape(z)[0]   #told T is sequence of observations, z are the observations here
        n = len(self.pi)     #need n for making B
        B = np.ones((T, n))  #beta is shape (T, n)
        
        #get last value for Beta: line 2 in psuedo code, want last val bc going bward here!
        B[-1] = c[-1]*np.ones(n)  #know want last val of B bc want B_T-1 and T-1 is last one!
        
        #do lines 3-6 of pseudo code: loop through T's
        for t in range(T-2, -1, -1):  #want start at T-2 and go down to 0!
            B[t] = np.sum(self.A.T*B[t+1], axis = 1)*self.B[z[t+1]] #compute B_t
            B[t] = np.array([np.sum([self.A[i,j]*B[t+1,i]*self.B[z[t+1], i] for i in range(n)]) for j in range(n)])
            B[t] = c[t]*B[t]          #rescale Beta
            
        return B   #want return Beta
    
    # Problem 4
    def _xi(self, z, alpha, beta, c):
        """
        Compute the xi and gamma probabilities.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix from the forward pass
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix from the backward pass
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        #do the probability formulas for xi and gamma given in lab manual (returning these 2 things)
        T = len(z)                 #told z has shape T
        m, n = self.B.shape  
        xi = np.zeros((T-1, n, n)) #told xi has this shape so initialize it like so
        gamma = np.zeros((T, n))   #told gamma has this shape so initialize it like so
        
        #now get xi and gamma probabilities!
        xi = np.array([alpha[t].reshape(-1, 1) * self.A.T * self.B[z[t+1], :] * beta[t+1, :] for t in range(T-1)])
        gamma = ((alpha*beta).T/c).T
        
        return xi, gamma
    
    # Problem 5
    def _estimate(self, z, xi, gamma):
        """
        Estimate better parameter values and update self.pi, self.A, and
        self.B in place.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        self.pi = gamma[0]   #update self.pi in place and this is update formula for pi given in lab manual
        
        T = len(z)
        I, J = self.B.shape  #will use for i,j in B's update formula
        
        #update self.A and self.B using update formulas given in lab manual:
        self.A = (np.sum(xi, axis = 0) / np.sum(gamma[:-1], axis = 0).reshape(-1,1)).T
        
        #know have if else for delta_zt so get that first and then use it to update B
        delta_ting = lambda i, j: np.sum([1*gamma[t,j] if z[t] == i else 0 for t in range(T)])
        self.B = np.array([[delta_ting(i,j)/np.sum(gamma[:,j]) for j in range(J)] for i in range(I)])
    
    # Problem 6
    def fit(self, z, pi, A, B, max_iter=100, tol=1e-5):
        """
        Fit the model parameters to a given observation sequence.

        Parameters
        ----------
        z : ndarray of shape (T,)
            Observation sequence on which to train the model.
        pi : Stochastic ndarray of shape (n,)
            Initial state distribution
        A : Stochastic ndarray of shape (n, n)
            Initial state transition matrix
        B : Stochastic ndarray of shape (m, n)
            Initial state observation matrix
        max_iter : int
            The maximum number of iterations to take
        tol : float
            The convergence threshold for change in log-probability
        """
        #follow the HMM fitting algorithm psuedo code given:
        
        #initialize self.pi, self.A, self.B
        self.pi = pi
        self.A = A
        self.B = B
        
        #compute logP(z|theta)
        alpha, c = self._forward(z)  #told line 6 to run forward pass, but need 1st for getting probability
        prob_z = -np.sum(np.log(c))
        
        # run the iteration, lines 6-13 in pseudo code
        for i in range(max_iter):
            beta_hat = self._backward(z, c)              #run bward pass
            xi, gamma = self._xi(z, alpha, beta_hat, c)  #compute xi, gamma prob.
            self._estimate(z, xi, gamma)                 #update model parameters (means call prob 5)
            alpha, c = self._forward(z)                  #need for finding log prob. on next line     
            new_prob_z = -np.sum(np.log(c))              #compute logP(z|theta) using newly updated parameters
            
            #if change in log prob. less than eps then break
            if np.abs(prob_z - new_prob_z) < tol:
                break
            else:
                prob_z = new_prob_z  #update the prob to be the new one and continue!
                

In [3]:
# TESTING STUFF IN THIS CELL !!!

# toy HMM example to be used to check answers
pi = np.array([.6, .4])
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
z = np.array([0, 1, 0, 2])

#test prob 2:
h = HMM()
h.pi = pi
h.A = A
h.B = B
alpha, c = h._forward(z)
print("prob 2 test answer: " + str(-np.log(c).sum())) # the log prob of observation

print("")
#test prob 3:
beta = h._backward(z, c)
print("prob 3 test answer: " + str(beta))

print("")
#test prob 4:
xi, gamma = h._xi(z, alpha, beta, c)
print("prob 4 test answer: " + str(xi))
print("")
print("still prob 4 tests answer: " + str(gamma))

#test prob 5:
h._estimate(z, xi, gamma)

print("")
print("prob 5 test answer: " + str(h.pi))
print("")
print("prob 5 test answer: " + str(h.A))
print("")
print("prob 5 test answer: " + str(h.B))

prob 2 test answer: -4.642913590898749

prob 3 test answer: [[3.1361635  2.89939354]
 [2.86699344 4.39229044]
 [3.898812   2.66760821]
 [3.56816483 3.56816483]]

prob 4 test answer: [[[0.14166321 0.0465066 ]
  [0.37776855 0.43406164]]

 [[0.17015868 0.34927307]
  [0.05871895 0.4218493 ]]

 [[0.21080834 0.01806929]
  [0.59317106 0.17795132]]]

still prob 4 tests answer: [[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]

prob 5 test answer: [0.18816981 0.81183019]

prob 5 test answer: [[0.55807991 0.49898142]
 [0.44192009 0.50101858]]

prob 5 test answer: [[0.23961928 0.70056364]
 [0.29844534 0.21268397]
 [0.46193538 0.08675238]]


In [4]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case

    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)

    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))

    # convert the data to a NumPy array of symbols
    a = np.array(list(data))

    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}

    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)

    return symbols, obs_sequence

## Problem 7

Train a HMM on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values ($26$ lower case characters and $1$ whitespace character), and run for $150$ iterations with the deffault value for `tol`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [5]:
symbols, obs = prep_data('declaration.txt') #apply helper code given above to declaration.txt file

M = len(set(obs))  #told to do this in lab manual, want 27 observations
N = 2              #want 2 states
max_iter = 150     #want run 150 iterations

#given these 3 lines of code next in lab manual after do M=len(set(obs)) so do it here since we have that line
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

h = HMM()          #initialize the HMM object

h.fit(obs, pi, A, B, max_iter) #now fit it!

for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}"
          .format(symbols[i], h.B[i,0], h.B[i,1]))

 , 0.1535, 0.1866
a, 0.0619, 0.0592
b, 0.0000, 0.0347
c, 0.0301, 0.0112
d, 0.0495, 0.0001
e, 0.1306, 0.0710
f, 0.0000, 0.0657
g, 0.0256, 0.0000
h, 0.0686, 0.0000
i, 0.0883, 0.0000
j, 0.0000, 0.0058
k, 0.0028, 0.0000
l, 0.0035, 0.0768
m, 0.0152, 0.0244
n, 0.0950, 0.0000
o, 0.0304, 0.1308
p, 0.0000, 0.0504
q, 0.0000, 0.0022
r, 0.0234, 0.1117
s, 0.0684, 0.0474
t, 0.1156, 0.0191
u, 0.0053, 0.0664
v, 0.0146, 0.0000
w, 0.0156, 0.0065
x, 0.0014, 0.0006
y, 0.0000, 0.0295
z, 0.0008, 0.0000


## Problem 8

Repeat the previous calculation with $3$ hidden states and again with $4$ hidden states. Interpret/explain your results.

In [6]:
#repeat prob 7 w/ 3 hidden states
N = 3

pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

h = HMM()

h.fit(obs, pi, A, B, max_iter) #now fit it!

for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}"
.format(symbols[i], h.B[i,0], h.B[i,1], h.B[i,2]))

 , 0.0306, 0.1258, 0.3106
a, 0.0000, 0.0697, 0.1100
b, 0.0146, 0.0278, 0.0000
c, 0.0398, 0.0375, 0.0000
d, 0.0848, 0.0074, 0.0009
e, 0.0000, 0.0000, 0.2780
f, 0.0546, 0.0147, 0.0000
g, 0.0413, 0.0080, 0.0000
h, 0.1168, 0.0000, 0.0082
i, 0.0000, 0.0320, 0.1249
j, 0.0058, 0.0000, 0.0000
k, 0.0040, 0.0006, 0.0006
l, 0.0601, 0.0311, 0.0001
m, 0.0407, 0.0159, 0.0000
n, 0.0457, 0.1813, 0.0000
o, 0.0000, 0.0832, 0.1131
p, 0.0289, 0.0295, 0.0000
q, 0.0022, 0.0000, 0.0000
r, 0.1112, 0.0596, 0.0000
s, 0.1125, 0.0791, 0.0035
t, 0.1351, 0.1352, 0.0000
u, 0.0000, 0.0411, 0.0415
v, 0.0267, 0.0000, 0.0000
w, 0.0217, 0.0187, 0.0000
x, 0.0018, 0.0021, 0.0000
y, 0.0196, 0.0000, 0.0086
z, 0.0014, 0.0000, 0.0000


In [7]:
#repeat prob 7 w/ 4 hidden states
N = 4

pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

h = HMM()

h.fit(obs, pi, A, B, max_iter) #now fit it!

#change this for 4 hidden states
for i in range(len(h.B)):
     print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}, {4:0.4f}"
.format(symbols[i], h.B[i,0], h.B[i,1], h.B[i,2], h.B[i,3]))

 , 0.0000, 0.2689, 0.0000, 0.5866
a, 0.0000, 0.0128, 0.2062, 0.0024
b, 0.0244, 0.0205, 0.0000, 0.0000
c, 0.0453, 0.0429, 0.0000, 0.0000
d, 0.0956, 0.0038, 0.0000, 0.0000
e, 0.0077, 0.0000, 0.3561, 0.0384
f, 0.0614, 0.0139, 0.0000, 0.0000
g, 0.0486, 0.0031, 0.0000, 0.0000
h, 0.0320, 0.0000, 0.0000, 0.1799
i, 0.0000, 0.0224, 0.1190, 0.1029
j, 0.0062, 0.0000, 0.0000, 0.0000
k, 0.0045, 0.0000, 0.0011, 0.0000
l, 0.0547, 0.0381, 0.0000, 0.0183
m, 0.0402, 0.0258, 0.0000, 0.0000
n, 0.0214, 0.2721, 0.0000, 0.0000
o, 0.0000, 0.0000, 0.2270, 0.0096
p, 0.0420, 0.0189, 0.0000, 0.0000
q, 0.0023, 0.0000, 0.0000, 0.0000
r, 0.0760, 0.1048, 0.0000, 0.0436
s, 0.1215, 0.0910, 0.0101, 0.0000
t, 0.2413, 0.0096, 0.0000, 0.0026
u, 0.0000, 0.0346, 0.0597, 0.0158
v, 0.0287, 0.0000, 0.0000, 0.0000
w, 0.0311, 0.0108, 0.0000, 0.0000
x, 0.0000, 0.0057, 0.0000, 0.0000
y, 0.0135, 0.0003, 0.0208, 0.0000
z, 0.0016, 0.0000, 0.0000, 0.0000


## Problem 9

Repeat the same calculation with `WarAndPeace.txt` for $2$ and $3$ hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [8]:
#repeat prob 7 and 8 but with different file
symbols, obs = prep_data("WarAndPeace.txt")

N = 2              #want do w/ 2 hidden states
M = len(set(obs))
max_iter = 150

pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

h = HMM()

h.fit(obs, pi, A, B, max_iter) #now fit it!

for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}"
          .format(symbols[i], h.B[i,0], h.B[i,1]))

 , 0.2146, 0.0877
а, 0.0000, 0.1759
б, 0.0250, 0.0000
в, 0.0655, 0.0000
г, 0.0296, 0.0000
д, 0.0385, 0.0001
е, 0.0180, 0.1427
ж, 0.0140, 0.0000
з, 0.0252, 0.0000
и, 0.0016, 0.1315
й, 0.0149, 0.0000
к, 0.0497, 0.0010
л, 0.0719, 0.0000
м, 0.0381, 0.0000
н, 0.0973, 0.0000
о, 0.0000, 0.2407
п, 0.0347, 0.0062
р, 0.0597, 0.0000
с, 0.0513, 0.0280
т, 0.0780, 0.0000
у, 0.0000, 0.0590
ф, 0.0018, 0.0003
х, 0.0111, 0.0000
ц, 0.0049, 0.0000
ч, 0.0167, 0.0038
ш, 0.0109, 0.0000
щ, 0.0047, 0.0000
ъ, 0.0003, 0.0003
ы, 0.0000, 0.0376
ь, 0.0009, 0.0434
э, 0.0000, 0.0066
ю, 0.0079, 0.0024
я, 0.0128, 0.0328
ё, 0.0000, 0.0001


In [9]:
#repeat prob 7 and 8 but with different file
symbols, obs = prep_data("WarAndPeace.txt")

N = 3            #want do w/ 3 hidden states now
M = len(set(obs))
max_iter = 150

pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

h = HMM()

h.fit(obs, pi, A, B, max_iter) #now fit it!

for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}"
.format(symbols[i], h.B[i,0], h.B[i,1], h.B[i,2]))

 , 0.0337, 0.4148, 0.0725
а, 0.1945, 0.0000, 0.0000
б, 0.0000, 0.0123, 0.0341
в, 0.0000, 0.0504, 0.0721
г, 0.0000, 0.0173, 0.0376
д, 0.0000, 0.0262, 0.0456
е, 0.1545, 0.0388, 0.0000
ж, 0.0000, 0.0060, 0.0198
з, 0.0000, 0.0204, 0.0268
и, 0.1480, 0.0000, 0.0000
й, 0.0000, 0.0291, 0.0000
к, 0.0000, 0.0352, 0.0589
л, 0.0000, 0.0373, 0.0959
м, 0.0000, 0.0275, 0.0436
н, 0.0000, 0.0406, 0.1390
о, 0.2661, 0.0000, 0.0000
п, 0.0000, 0.0248, 0.0473
р, 0.0000, 0.0165, 0.0931
с, 0.0000, 0.0952, 0.0384
т, 0.0000, 0.0357, 0.1084
у, 0.0647, 0.0007, 0.0000
ф, 0.0000, 0.0017, 0.0021
х, 0.0000, 0.0086, 0.0120
ц, 0.0000, 0.0009, 0.0082
ч, 0.0000, 0.0164, 0.0198
ш, 0.0000, 0.0046, 0.0155
щ, 0.0000, 0.0002, 0.0084
ъ, 0.0000, 0.0000, 0.0009
ы, 0.0415, 0.0000, 0.0000
ь, 0.0494, 0.0000, 0.0000
э, 0.0073, 0.0000, 0.0000
ю, 0.0033, 0.0147, 0.0000
я, 0.0369, 0.0242, 0.0000
ё, 0.0001, 0.0000, 0.0000
